In [1]:
import sys
sys.path.append('../../src')
from config import Paths

In [2]:
import pickle
import pprint
import pandas as pd
import pickle

langs = ["all", "EN", "IT"]

df = pd.read_csv(
    Paths.testnino1_classification_task,
    usecols=["id", "Lang", "Data", "Link", "Titolo", "Testo", "Stato Workflow"],
)

df["Data"] = pd.to_datetime(df["Data"])

target_column = "Stato Workflow"
# remove rows not validated by the analyst
df = df[df[target_column] != "N"]

headers = {}
datas = {}

for lang in langs:
    datas[lang] = []
    if lang == "all":
        df_lang = df
    else:
        df_lang = df[df["Lang"] == lang]
    tot = df_lang.shape[0]
    Y_count = df_lang[df_lang["Stato Workflow"] == "Y"].shape[0]
    D_count = df_lang[df_lang["Stato Workflow"] == "D"].shape[0]

    headers[lang] = [["tot", "Y_count", "D_count"], [tot, Y_count, D_count]]

    # Load the dictionaries from the file
    with open(f"results_{lang}.pkl", "rb") as file:
        cms = pickle.load(file)
        accuracies = pickle.load(file)
        fit_times = pickle.load(file)
        prediction_times = pickle.load(file)

        for key, value in accuracies.items():
            data = []
            key_parts = key.split("-", 2)
            if len(key_parts) >= 3:
                train_size = key_parts[0]
                classifier_name = key_parts[1]
                hyperparams = key_parts[2]

                data.append(classifier_name)
                data.append(train_size)
                data.append(round(value, 2))
            
            for cm_value in cms[key].flatten().tolist():
                data.append(cm_value)
            
            data.append(round(fit_times[key],2))
            data.append(round(prediction_times[key],2))
            data.append(f'"{hyperparams}"')
            datas[lang].append(data)

pprint.pprint(datas)


{'EN': [['GradientBoostingClassifier',
         '25',
         0.55,
         674,
         572,
         453,
         601,
         0.22,
         0.24,
         '"{\'ccp_alpha\': 0.0, \'criterion\': \'friedman_mse\', \'init\': '
         "None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, "
         "'max_features': None, 'max_leaf_nodes': None, "
         "'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, "
         "'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, "
         "'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 42, "
         "'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, "
         '\'verbose\': 0, \'warm_start\': False}"'],
        ['GradientBoostingClassifier',
         '50',
         0.63,
         704,
         518,
         325,
         702,
         0.35,
         0.22,
         '"{\'ccp_alpha\': 0.0, \'criterion\': \'friedman_mse\', \'init\': '
         "None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_

In [3]:
import csv

for lang in langs:

    output_path = f'output_{lang}.csv'

    with open(output_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer = csv.writer(file)
        writer.writerows(headers[lang])
        writer.writerows(datas[lang])